In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("spotify_millsongdata.csv")

In [4]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [5]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [6]:
df.shape

(57650, 4)

In [7]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [8]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [9]:
df.head(10)

,artist,song,text
0,Carpenters,Touch Me When We're Dancing,Play us a song we can slow dance on \r\nWe wa...
1,Loretta Lynn,Back In Baby's Arms,I'm back in baby's arms how I've missed those ...
2,Pet Shop Boys,In The Night (1995),"Zazou, whatcha gonna do? \r\nThere's a lot of..."
3,Patsy Cline,It Wasn't God Who Made Honky Tonk Angels,It wasn't God who made honky tonk angels \r\n...
4,Utopia,Zen Machine,You're always finding out \r\nAbout the thing...
5,Santana,Stone Flower,What's that sound I hear coming from the groun...
6,Bob Seger,Always In My Heart,I think of you and I together \r\nI picture y...
7,Frank Zappa,"Carol, You Fool","Frank zappa (lead guitar, vocals) \r\nIke wil..."
8,Air Supply,Crazy Love,"When you are young, the thought of the day is ..."
9,Madonna,Each Time You Break My Heart,Each time you break my heart \r\nEach time yo...


In [10]:
df['text'][0]

"Play us a song we can slow dance on  \r\nWe want to hold each other  \r\nPlay us a groove so we hardly move  \r\nJust let our hearts beat together  \r\nOh baby, 'cause it feel so good  \r\nWhen we're close like this  \r\nWhisper is my ear and let me steal a kiss  \r\n  \r\nCome on and touch me  \r\nWhen we're dancing  \r\nYou know you've got that loving touch  \r\n(Loving touch)  \r\nTouch me when we're dancing  \r\nI want to feel you when I'm falling in love  \r\n  \r\nTonight's the night and it feels so right  \r\nMy heart is saying it to me  \r\nYou're the one I've waited for so long  \r\nSo let your love flow through me  \r\nOh baby, 'cause it feels so good  \r\nJust to be this close  \r\nYou've got me up so high  \r\nI could fly coast to coast\r\n\r\n"

In [11]:
# df = df.sample(5000)

In [12]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [13]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [14]:
%pip install nltk
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [15]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [18]:
similarity[0]

array([1.        , 0.08047804, 0.0492408 , ..., 0.01612109, 0.02358081,
       0.06236531])

In [19]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [20]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [22]:
if 'Crying Over You' in df['song'].values:
    recommendation('Crying Over You')
else:
    print("The song 'Crying Over You' does not exist in the dataframe.")


The song 'Crying Over You' does not exist in the dataframe.


In [23]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))